In [1]:
from itertools import count
import numpy as np

In [2]:
def data(n,d): #Generating Data
    mean=np.linspace(0,1,d)
    covar=np.identity(d)
    x=np.random.multivariate_normal(mean,covar,n).T
    return x

The code takes dimensions of samples (d) , number of samples (N) , Number of mixtures (K), and threshold for stopping condition (e) as input from the end user.

Values used to demonstrate output are d = 1, N = 1000, K = 4, e = 0.01

In [3]:
x = data(1000,2)#CHeck
print(x)  

[[-0.67948301 -0.48747073  0.91225269 ...  0.73762224 -1.11075031
  -0.56730619]
 [ 0.5874291   0.75145621  0.08291472 ...  1.16895238  1.80031152
   0.0853821 ]]


In [4]:
def read(): #Reading Inputs
    d = int(input("Enter Dimension of Samples: "))  #1
    N = int(input("Enter Number of Samples: "))  #1000
    K = int(input("Enter K value: ")) #4
    e = float(input("Enter Threshold Value: ")) #0.01
    return d, N, K, e

In [13]:
def write(u, pi, E, likelihoods, deltas): #Printing Results
    stop = len(likelihoods)
    for i in range(stop):
        print(f"Likelihood {i}: {likelihoods[i]}")
        print(f"Error {i}: {deltas[i]}")
    print("\n")
    print(f"Estimated Means: {u}")
    print(f"Estimated Weights: {pi}")
    print(f"Estimated Covariance: {E}")

In [6]:
def multigauss(X, mean, covar):
    dr = np.sqrt(((2 * np.pi) ** (X.shape[0]) * np.linalg.det(covar)))
    nr = np.matmul((X - mean), np.linalg.inv(covar))
    nr = np.matmul((X - mean), nr)
    nr = np.exp(-1 / 2 * (nr))
    pdf = nr / dr
    return pdf

In [7]:
def log_likelihood(x, mean, weights, covar, K, N): 
    #Mean - mu | Weights - Pi | Covar - E
    Lx = 0
    for i in range(N):
        Ll = 0
        for j in range(K):
            Ll += weights[j] * multigauss(x[:, i], mean[:, j], covar[j])
        Lx += np.log(Ll)
    return Lx

In [8]:
def params(d, K): #Assigning Parameters from Hyperparameters
    mean = np.random.rand(d, K)
    weights = (1.0 / K) * np.ones(K)
    covar = [np.identity(d)] * K
    return covar, mean, weights   

In [9]:
a,b,c = params(2,4)  #Check
a = np.array(a)
b = np.array(b)
c = np.array(c)
print(a.shape)
print(b.shape)
print(c.shape)

(4, 2, 2)
(2, 4)
(4,)


In [10]:
def updated_params(N, K, d, X, mean, covar , weights): #Updating the theta set
    pdf = np.zeros((N, K))
    gamma = np.zeros((N, K))
    mean_new = np.zeros((d, K))
    covar_new = np.zeros((K, d, d))
    
    for i in range(N):
        for j in range(K):
            pdf[i, j] = multigauss(X[:, i], mean[:, j], covar[j])

    
    for i in range(N): #Gamma 
        Nk = np.matmul(weights.T, pdf[i])
        for j in range(K):
            gamma[i, j] = weights[j] * pdf[i, j] / Nk
    
    N_K = sum(gamma)
    weights_new = N_K / N #New Weights
    
    for j in range(K): #New Mean
        mean_new[:, j] = np.matmul(X, gamma[:, j]) / N_K[j]
    
    
    
    for j in range(K):
        for i in range(N): #New Covariance
            c = X[:,i]-mean_new[:,j]
            covar_new[j] = covar_new[j] + gamma[i, j] * ((np.matrix(c.T) * np.matrix(c)))
        covar_new[j] = covar_new[j] / N_K[j]
        
    return mean_new, weights_new, covar_new

In [11]:
d, N, _, _ = read() #Printing Initial Parameters
covar, mean, weights = params(1,4)
print("Initial Parameters")
print("Mean :", mean)
print("Covariance :", covar)
print("Weights :", weights)

Enter Dimension of Samples: 1
Enter Number of Samples: 1000
Enter K value: 4
Enter Threshold Value: 0.01
Initial Parameters
Mean : [[0.36788331 0.27535329 0.90370539 0.50843934]]
Covariance : [array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]])]
Weights : [0.25 0.25 0.25 0.25]


In [14]:
def GMM(d, N, K, e):
    covar, mean, weights = params(d, K)
    X = data(N, d)
    t = 0
    err = 1 + e
    likelihoods, deltas = [], [] 
    for i in count(0):
        if err <= e:
            break
        lx1 = log_likelihood(X, mean, weights, covar, K, N)
        mean_new, weights_new, covar_new = updated_params(N, K, d, X, mean, covar, weights)
        lx2 = log_likelihood(X, mean_new, weights_new, covar_new, K, N)
        mean, weights, covar = mean_new, weights_new, covar_new
        err = lx2-lx1
        print("Iteration : %d"%i)
        print("Mean")
        print(mean)
        print("Covariance")
        print(covar)
        print("Weights")
        print(weights)
        print("\n")
        likelihoods.append(lx2)
        deltas.append(err)
    return mean, weights, covar, likelihoods,deltas


def main():
    print("Please Enter Same Parameters as entered in the cell above")
    d, N, K, e = read()
    mean, weights, covar, likelihoods, deltas = GMM(d, N, K, e)
    write(mean, weights, covar, likelihoods, deltas)


if __name__ == "__main__":
    main()

Please Enter Same Parameters as entered in the cell above
Enter Dimension of Samples: 1
Enter Number of Samples: 1000
Enter K value: 4
Enter Threshold Value: 0.01
Iteration : 0
Mean
[[ 0.12792141  0.42923583 -0.06180636 -0.28478225]]
Covariance
[[[0.92824365]]

 [[0.91328188]]

 [[0.94225354]]

 [[0.96204751]]]
Weights
[0.23904037 0.20267175 0.26372879 0.29455909]


Iteration : 1
Mean
[[ 0.12963377  0.42607407 -0.05969616 -0.28607774]]
Covariance
[[[0.91861691]]

 [[0.89302776]]

 [[0.9445283 ]]

 [[0.98275946]]]
Weights
[0.23910653 0.20272817 0.26367963 0.29448567]


Iteration : 2
Mean
[[ 0.13128033  0.42361713 -0.05778089 -0.28763651]]
Covariance
[[[0.9095481 ]]

 [[0.87658014]]

 [[0.94560512]]

 [[1.00088014]]]
Weights
[0.23915721 0.20280216 0.26361241 0.29442822]


Iteration : 3
Mean
[[ 0.13281819  0.4216128  -0.05601167 -0.28928624]]
Covariance
[[[0.90110501]]

 [[0.8632465 ]]

 [[0.94584043]]

 [[1.01678126]]]
Weights
[0.23920036 0.20288304 0.26353661 0.29437999]


Iteration : 4